In [1]:
%load_ext tensorboard
import tensorflow as tf
import datetime

In [2]:
# clear logs
!rm -rf ./logs/history_plotter1/

In [3]:
# load data

(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255., x_test / 255.

In [4]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

In [5]:
# build model

model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(10))

In [6]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

# Define our metrics
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [7]:
def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        preds = model(inputs, training=True)
        loss = loss_object(targets, preds)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    train_loss(loss)
    train_accuracy(targets, preds)


In [8]:
def test_step(inputs, targets):
    preds = model(inputs)
    loss = loss_object(targets, preds)

    test_loss(loss)
    test_accuracy(targets, preds)

In [9]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/history_plotter1/' + current_time + '/train'
test_log_dir = 'logs/history_plotter1/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [10]:
epochs = 5
for epoch in range(epochs):
    # Reset metrics every epoch
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()
    
    for (inp, tar) in train_dataset:
        train_step(inp, tar)
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

    for (inp, tar) in test_dataset:
        test_step(inp, tar)
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
  
    template = "Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}"
    print (
        template.format(
        epoch+1,
        train_loss.result(), 
        train_accuracy.result()*100,
        test_loss.result(), 
        test_accuracy.result()*100)
    )



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 0.247211292386055, Accuracy: 92.78666687011719, Test Loss: 0.12537232041358948, Test Accuracy: 96.26000213623047
Epoch 2, Loss: 0.1052914634346962, Accuracy: 96.8550033569336, Test Loss: 0.08240221440792084, Test Accuracy: 97.43000030517578
Epoch 3, Loss: 0.07305137068033218, Accuracy: 97.74333190917969, Test Loss: 0.07073584944009781, Test Accuracy: 97.70999908447266
Epoch 4, Loss: 0.05560490861535072, Accuracy: 98.2683334350586, Test Loss: 0.06372775882482529, Test Accuracy: 97.91999816

In [11]:
%tensorboard --logdir logs/history_plotter1/